In [19]:
EVENT_HUB_CONN_STR="Endpoint=sb://factored-datathon.servicebus.windows.net/;SharedAccessKeyName=datathon_group_1;SharedAccessKey=2GETvVt0FxyM0bo0Qau4inlmC/w3t4Uut+AEhAnAEgk=;EntityPath=factored_datathon_amazon_reviews_1"
EVENTHUB_NAME="factored_datathon_amazon_review_1"
CONSUMER_GROUP="datacticos"

Sync Connection test

In [ ]:
import os
from azure.eventhub import EventHubConsumerClient


consumer_client = EventHubConsumerClient.from_connection_string(
    conn_str=EVENT_HUB_CONN_STR,
    consumer_group=CONSUMER_GROUP,
    eventhub_name=EVENTHUB_NAME,
)

with consumer_client:
    # consumer_client is now ready to be used.
    consumer_client.get_eventhub_properties()

In [ ]:
import logging
from azure.eventhub import EventHubConsumerClient

client = EventHubConsumerClient.from_connection_string(
    conn_str=EVENT_HUB_CONN_STR,
    consumer_group=CONSUMER_GROUP,
    eventhub_name=EVENTHUB_NAME,
    logging_enable=False
)

logger = logging.getLogger("azure.eventhub").setLevel(logging.NOTSET)
logging.basicConfig(level=logging.INFO)

def on_event(partition_context, event):
    logger.info("Received event from partition {}".format(partition_context.partition_id))
    partition_context.update_checkpoint(event)
    print("Received event from partition {}".format(partition_context.partition_id))

with client:
    client.receive(
        on_event=on_event,
        starting_position="-1",  # "-1" is from the beginning of the partition.
    )
    # receive events from specified partition:
    # client.receive(on_event=on_event, partition_id='0')

INFO:azure.eventhub._eventprocessor.event_processor:EventProcessor '006a2c1a-2093-41ea-a036-766ebe82c8fb' is being started
INFO:azure.eventhub._pyamqp._connection:Connection state changed: None -> <ConnectionState.START: 0>
INFO:azure.eventhub._pyamqp._connection:Connection state changed: <ConnectionState.START: 0> -> <ConnectionState.HDR_SENT: 2>
INFO:azure.eventhub._pyamqp._connection:Connection state changed: <ConnectionState.HDR_SENT: 2> -> <ConnectionState.HDR_SENT: 2>
INFO:azure.eventhub._pyamqp._connection:Connection state changed: <ConnectionState.HDR_SENT: 2> -> <ConnectionState.OPEN_PIPE: 4>
INFO:azure.eventhub._pyamqp.session:Session state changed: <SessionState.UNMAPPED: 0> -> <SessionState.BEGIN_SENT: 1>
INFO:azure.eventhub._pyamqp.link:Link state changed: <LinkState.DETACHED: 0> -> <LinkState.ATTACH_SENT: 1>
INFO:azure.eventhub._pyamqp.management_link:Management link receiver state changed: <LinkState.DETACHED: 0> -> <LinkState.ATTACH_SENT: 1>
INFO:azure.eventhub._pyamqp.

KeyboardInterrupt: 

INFO:azure.eventhub._client_base:'eventhub.pysdk-41c7e778' has an exception (AuthenticationError('CBS Token authentication failed.\nStatus code: None\nError: client-error\nCBS Token authentication failed.\nStatus code: None')). Retrying...
INFO:azure.eventhub._pyamqp.link:Link state changed: <LinkState.ATTACHED: 3> -> <LinkState.DETACH_SENT: 4>
INFO:azure.eventhub._pyamqp.management_link:Management link receiver state changed: <LinkState.ATTACHED: 3> -> <LinkState.DETACH_SENT: 4>
INFO:azure.eventhub._pyamqp.link:Link state changed: <LinkState.ATTACHED: 3> -> <LinkState.DETACH_SENT: 4>
INFO:azure.eventhub._pyamqp.management_link:Management link sender state changed: <LinkState.ATTACHED: 3> -> <LinkState.DETACH_SENT: 4>
INFO:azure.eventhub._pyamqp.session:Session state changed: <SessionState.MAPPED: 3> -> <SessionState.END_SENT: 4>
INFO:azure.eventhub._pyamqp._connection:Connection state changed: <ConnectionState.OPENED: 9> -> <ConnectionState.CLOSE_SENT: 11>
INFO:azure.eventhub._pyamqp.

Async Connection test

In [25]:
import logging
import asyncio
from azure.eventhub.aio import EventHubConsumerClient

import nest_asyncio
nest_asyncio.apply()

logger = logging.getLogger("azure.eventhub")
logging.basicConfig(level=logging.INFO)

async def on_event(partition_context, event):
    logger.info("Received event from partition {}".format(partition_context.partition_id))
    print("Received event from partition {}".format(partition_context.partition_id))
    await partition_context.update_checkpoint(event)

async def receive():
    client = consumer_client = EventHubConsumerClient.from_connection_string(conn_str=EVENT_HUB_CONN_STR,consumer_group=CONSUMER_GROUP,eventhub_name=EVENTHUB_NAME,)
    async with client:
        await client.receive(
            on_event=on_event,
            starting_position="-1",  # "-1" is from the beginning of the partition.
        )
        # receive events from specified partition:
        # await client.receive(on_event=on_event, partition_id='0')

if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    loop.run_until_complete(receive())

INFO:azure.eventhub.aio._eventprocessor.event_processor:EventProcessor '49063595-389d-4db8-a384-0ac90ef37903' is being started
INFO:azure.eventhub._pyamqp.aio._connection_async:Connection state changed: None -> <ConnectionState.START: 0>
INFO:azure.eventhub._pyamqp.aio._connection_async:Connection state changed: <ConnectionState.START: 0> -> <ConnectionState.HDR_SENT: 2>
INFO:azure.eventhub._pyamqp.aio._connection_async:Connection state changed: <ConnectionState.HDR_SENT: 2> -> <ConnectionState.HDR_SENT: 2>
INFO:azure.eventhub._pyamqp.aio._connection_async:Connection state changed: <ConnectionState.HDR_SENT: 2> -> <ConnectionState.OPEN_PIPE: 4>
INFO:azure.eventhub._pyamqp.aio._session_async:Session state changed: <SessionState.UNMAPPED: 0> -> <SessionState.BEGIN_SENT: 1>
INFO:azure.eventhub._pyamqp.aio._link_async:Link state changed: <LinkState.DETACHED: 0> -> <LinkState.ATTACH_SENT: 1>
INFO:azure.eventhub._pyamqp.aio._management_link_async:Management link receiver state changed: <Link